# Init

In [ ]:
import numpy as np
from numpy.linalg import inv, norm, eig
from math import atan, sin, cos
import matplotlib.pyplot as plt
import random
import networkx as nx
import warnings

# Main

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
class web_graph_generator:
    def __init__(self):
        pass

    def generate_web_graph(self, size=5, show=False):
        matrix = np.zeros(shape=(size, size))

        for i in range(size):
            for j in range(size):
                matrix[i][j] = 1 if random.randint(0, 99) < 20 else 0
                if i == j:
                    matrix[i][j] = 0
        matrix = self.additional_edges(matrix)
        if show :
            print(matrix)
        return matrix

    def additional_edges(self, matrix):
        len_ = len(matrix)
        for i, row in enumerate(matrix):
            if sum(row) == 0:
                j = random.randint(0, len_ - 1)
                matrix[i][j if i != j else j+1 if i == 0 else j - 1] = 1

        for column in range(len_):
            if sum(matrix[:, column]) == 0:
                i = random.randint(0, len(matrix) - 1)
                matrix[i if i != column else i+1 if column == 0 else i - 1][column] = 1

        return matrix

    def draw_classic(self, matrix):
        web_graph = nx.DiGraph(matrix)
        nx.draw(web_graph, with_labels=True)

        return

    def draw_size_dependent(self, matrix):
        web_graph = nx.DiGraph(matrix)
        # print(nx.degree(web_graph))
        weights = [v*200 for c, v in nx.degree(web_graph)]
        all_weights = sum(weights)
        colors = [weights[i]/all_weights for i in range(len(weights))]
        nx.draw(web_graph, node_size=weights, with_labels=True, node_color = colors)

        return

    def draw_pagerank(self, matrix, weights):
        web_graph = nx.DiGraph(matrix)
        # print(nx.degree(web_graph))
        weights = [v*5*1e4 for v in weights]
        all_weights = sum(weights)
        colors = [weights[i]/all_weights for i in range(len(weights))]
        nx.draw(web_graph, node_size=weights, with_labels=True, node_color = colors)

        return

    def generate_matrix_A(self, matrix):
        len_ = len(matrix)
        A = np.zeros(shape=(len_, len_))
        for i in range(len_):
            from_vertex =sum(matrix[i])
            for j in range(len_):
                A[i][j] = matrix[i][j] / from_vertex

        return A

In [ ]:
class solver:
    def __init__(self):
        return

    def pagerank(self, matrix, alpha=0.85, accuracy=10**(-9)):
        # B = alpha * A + (1 - alpha) * M where M = (m_ij), mij = 1/n
        A = matrix.copy().T
        n = A.shape[0]
        for i in range(n):
            col = A[:, i]
            A[:, i] = col/np.count_nonzero (col)
        M = np.ones(A.shape) / n
        B = alpha * A + (1 - alpha) * M
        X_prev = np.zeros((n, 1))
        X_k = np.ones((n, 1)) / n

        while np.linalg.norm(X_k - X_prev) > accuracy:
            X_prev = X_k
            X_k = np.dot(B, X_k)

        result = X_k.reshape((1, -1))

        return result[0]

    @staticmethod
    def generate_random_matrix(size, min_value=0, max_value=100):
        matrix = np.random.randint(min_value, high=max_value + 1, size=(size, size))
        return matrix

    def generate_matrix_from_eigen(self, values, matrix):
        matrix = matrix.copy().T
        if (len(values) != matrix.shape[0]):
            return
        M = np.diag(np.array(values))
        S_inv = np.linalg.inv(matrix)
        return matrix @ M @ S_inv

    def generate_symmetric_matrix(self, size, min_value=0, max_value=10):
        a = np.random.randint(min_value, max_value, (size,size))
        result = np.tril(a) + np.tril(a, -1).T
        return result

    def stop_moment(self, matrix):
        x = [matrix[i, j] ** 2 for j in range(matrix.shape[0]) for i in range(matrix.shape[0]) if i != j]
        #print(x)
        return sum(x)

    def max_non_diagonal(self, matrix):
        n = len(matrix)
        aMax = 0.0
        for i in range(n-1):
            for j in range(i+1,n):
                if abs(matrix[i,j]) > aMax:
                    aMax = abs(matrix[i,j])
                    k = i; l = j
        return k,l

    def jacobi(self, matrix, eps=1e-4):
        it = 0
        a = matrix.copy()
        u_al = np.eye(a.shape[0])

        while self.stop_moment(a) >= eps:

            it += 1
            u = np.eye(a.shape[0])

            i, j = self.max_non_diagonal(a)

            phi = atan(2 * a[i, j] / (a[i,i] - a[j,j])) / 2
            u[i, [i, j]],u[j, [i, j]] = [cos(phi), - sin(phi)], [sin(phi), cos(phi)]

            u_al = u_al @ u
            a = u.T @ a @ u

            #print(f'\n Iteration {it}:  \n'
            #   f'\tt(matrix) = {stop_moment(a)}.\n')


        return np.diag(a), u_al

    def check_symmetric(self, a, rtol=1e-05, atol=1e-08):
        return np.allclose(a, a.T, rtol=rtol, atol=atol)

# Testing

In [ ]:
gen = web_graph_generator()

In [ ]:
solver = solver()

## PageRank

In [ ]:
matrix = gen.generate_web_graph(15)
matrix

In [ ]:
gen.draw_classic(matrix)

In [ ]:
gen.draw_size_dependent(matrix)

In [ ]:
matrix_A = gen.generate_matrix_A(matrix)
matrix_A

In [ ]:
pagerank_matrix = solver.pagerank(matrix)
print(*pagerank_matrix)

In [ ]:
print(*[prob for prob in nx.pagerank(nx.DiGraph(matrix)).values()])

In [ ]:
gen.draw_pagerank(matrix, pagerank_matrix)

## Jacobi eigenvalue algorithm

In [ ]:
test_matr = np.matrix([[2, -1, 0, 0],[-1, 2, -1, 0],[0, -1, 2, -1],[0,0, -1, 2]])
print(test_matr)

module_result = np.linalg.eig(test_matr)
print("np.linalg.eig:\n", *module_result[0])
print("np.linalg.eig EigenVectors:\n", module_result[1])

jacobi_result = solver.jacobi(test_matr)
print("Jacobi:\n", *jacobi_result[0])
print("Jacobi Eigenvectors:\n", jacobi_result[1])

In [ ]:
symetric = solver.generate_symmetric_matrix(size=5)
print(symetric)

module_result = np.linalg.eig(symetric)
print("np.linalg.eig:\n", *module_result[0])
print("np.linalg.eig EigenVectors:\n", module_result[1])
jacobi_result = solver.jacobi(symetric)
print("Jacobi:\n", *jacobi_result[0])
print("Jacobi Eigenvectors:\n", jacobi_result[1])


## N-eigenvalue + N-eigenvectors

In [ ]:
test_matrix = solver.generate_matrix_from_eigen([1,3,1,1,1], solver.generate_random_matrix(5))
test_matrix

In [ ]:
cnt = 0
for i in range(10000):
    try:
        eigen_value = np.random.randint(0, 10, size=3)
        test = solver.generate_matrix_from_eigen(eigen_value, solver.generate_random_matrix(3))
        if solver.check_symmetric(test) :
            cnt += 1
            # if cnt == 1:
            #     print(eigen_value)
            #     print(solver.jacobi(test))
    except:
        continue

print(cnt)